<a href="https://colab.research.google.com/github/Dpksngh27/Dpksngh27/blob/main/Next_word_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
text =  """Virat Kohli (Hindi pronunciation: born 5 November 1988) is an Indian international cricketer who currently plays Test
          cricket and ODI cricket for India. Kohli is a former T20I player and a former
          captain of the Indian national cricket team. He is a right-handed batsman and an
          occasional unorthodox right arm quick bowler. He currently represents Royal
          Challengers Bengaluru in the IPL and Delhi in domestic cricket. He holds the
          record as the highest run-scorer in IPL, ranks third in T20I, third in ODI, and
          stands as the fourth-highest in international cricket.[4] He also holds the
          record for scoring the most centuries in ODI cricket and stands second in the
          list of most international centuries scored. Hence, Kohli is widely regarded as
          one of the greatest batsmen of all time and the greatest batsman in the modern era.
           Kohli was a key member of the Indian team that won the 2011 Cricket World Cup,
           2013 Champions Trophy and 2024 T20 World Cup and captained India to win the ICC
           Test match three"""

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import torch

In [51]:
from torch.utils.data import Dataset

In [52]:
word2idx = {word: i for i, word in enumerate(set(text.split()))}
idx2word = {i:word for word, i in word2idx.items()}

In [53]:
class customDataset(Dataset):
  def __init__(self,text, sequence_length,wrord2idx):
    self.text = text
    self.sequence_length = sequence_length
    self.word2idx = word2idx

  def __len__(self):
    return len(self.text) - self.sequence_length

  def __getitem__(self, index):
    sequence = [self.word2idx[word] for word in self.text[index: index+self.sequence_length]]
    target = self.word2idx[self.text[index+self.sequence_length]]

    return torch.tensor(sequence), torch.tensor(target)

In [54]:
dataset = customDataset(text.split(),20,word2idx)

In [55]:
dataset[10]

(tensor([ 11,  92, 100,  83,  18,  74,  88,  24,  30,  53,  24,  73,  95,  56,
          19,  84,  46,  42,  16,  30]),
 tensor(84))

In [56]:
idx2word = {idx: word for word, idx in word2idx.items()}

In [57]:
from torch.utils.data import DataLoader

In [58]:
dataloader = DataLoader(dataset, batch_size=32, shuffle = True)

In [59]:
import torch.nn as nn

In [60]:
class LSTM(nn.Module):
  def __init__ (self, vocab_size, embed_size, hidden_size):
    super(LSTM, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size,vocab_size)

  def forward(self, x, h0,c0):
    embed = self.embed(x)
    out,(h_n, c_n) = self.lstm(embed, (h0,c0))
    output = self.fc(out[:,-1,:])
    return output, (h_n, c_n)

In [61]:
model = LSTM(len(word2idx), embed_size = 128, hidden_size=256)

In [62]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.05)
criterion = nn.CrossEntropyLoss()

# Training LSTM MOdel

In [63]:
for epoch in range(10):
  for input, label in dataloader:
    input = input
    label = label


    optimizer.zero_grad()
    h0 = torch.zeros(1, input.size(0), 256)
    c0 = torch.zeros(1, input.size(0),256)
    outputs,_ = model(input, h0, c0)
    loss = criterion(outputs, label)
    loss.backward()
    optimizer.step()
  print(f"Epoch{epoch} : Loss: {loss.item()}")


Epoch0 : Loss: 4.633436679840088
Epoch1 : Loss: 4.585517406463623
Epoch2 : Loss: 4.55507755279541
Epoch3 : Loss: 4.575130939483643
Epoch4 : Loss: 4.562908172607422
Epoch5 : Loss: 4.550714492797852
Epoch6 : Loss: 4.492986679077148
Epoch7 : Loss: 4.49875020980835
Epoch8 : Loss: 4.516615390777588
Epoch9 : Loss: 4.501417636871338


In [64]:
# input_seq = torch.tensor([word2idx[word] for word in text.split()]).unsqueeze(0)
# h0 = torch.zeros(10, input_seq.size(0), 256)
# c0 = torch.zeros(10, input_seq.size(0), 256)
# output, _ = model(input_seq, h0, c0)
# predicted_word = torch.argmaz(output).item()
# print(f"Predicted word: {idx2word[predicted_word]}")

In [65]:
input_seq = torch.tensor([word2idx[word] for word in text.split()]).unsqueeze(0)
h0 = torch.zeros(1, input_seq.size(0), 256)  # num_layers = 1, batch_size = 1, hidden_size = 256
c0 = torch.zeros(1, input_seq.size(0), 256)
output, _ = model(input_seq, h0, c0)
predicted_word = torch.argmax(output).item() # torch.argmax instead of torch.argmaz
print(f"Predicted word: {idx2word[predicted_word]}")

Predicted word: the


In [66]:
out, _ = model(input_seq, h0, c0)

In [67]:
out.shape

torch.Size([1, 102])